# Configure KBase Jupyter Dev Environment
<sub><sup>(contact chenry@anl.gov with questions)</sub></sup>

In [1]:
import platform
print("python version " + platform.python_version())
import sys
import json
from json import dump
import os
import pandas as pd
from pandas import DataFrame, read_csv, concat, set_option
from os.path import exists
from pathlib import Path
import logging
import shutil
from configparser import ConfigParser

config = ConfigParser()
if not exists(str(Path.home()) + '/.kbase/config'):    
    if exists("/scratch/shared/code/sharedconfig.cfg"):
        shutil.copyfile("/scratch/shared/code/sharedconfig.cfg",str(Path.home()) + '/.kbase/config')
    else:
        print("You much create a config file in ~/.kbase/config before running this notebook. See instructions: https://docs.google.com/document/d/1fQ6iS_uaaZKbjWtw1MgzqilklttIibNO9XIIJWgxWKo/edit")
        sys.exit(1)
config.read(str(Path.home()) + '/.kbase/config')
paths = config.get("DevEnv","syspaths").split(";")
codebase = config.get("DevEnv","codebase",fallback="")
for i,filepath in enumerate(paths):
    if filepath[0:1] != "/":
        paths[i] = codebase+"/"+filepath
sys.path = paths + sys.path

from chenry_utility_module.kbdevutils import KBDevUtils
kbdevutil = KBDevUtils("ModelSEED2")
from modelseedpy import AnnotationOntology, MSPackageManager, MSModelUtil, MSBuilder, MSATPCorrection, MSGapfill, MSModelReport
from modelseedpy.core.mstemplate import MSTemplateBuilder
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
msrecon = kbdevutil.msseedrecon()
msrecon.core_template = msrecon.get_template(msrecon.templates["core"],None)

annoapi = kbdevutil.anno_client(native_python_api=True)

com_media = msrecon.get_media("KBaseMedia/Complete")
gmm_media = msrecon.get_media("KBaseMedia/Carbon-D-Glucose")
auxo_media = msrecon.get_media("94026/Auxotrophy_media")

python version 3.9.13
KBBaseModules 0.0.1
Output files printed to:/Users/chenry/workspace/Notebooks//ModelSEED2//sessions/default/output when using KBDevUtils.output_dir
modelseedpy 0.3.3
cobrakbase 0.3.1


# Creating debuggable model

In [2]:
#Pulling super annotated ecoli genome
genome_ref = "77537/Eco_RAST_Prokka_BlastKOALA_PTools_DeepEC_DeepGO"
#Link to AnnotationOntology code in github:
annoont = AnnotationOntology.from_kbase_data(annoapi.get_annotation_ontology_events({
    "input_ref" : genome_ref
}),genome_ref)
gene_term_hash = annoont.get_gene_term_hash(prioritized_event_list=None,ontologies=["SSO"],merge_all=True,feature_type="gene")
#kbdevutil.save("gene_term_hash",gene_term_hash)

1707543694.832675 INFO: get_annotation_ontology_events:{
    "input_ref": "77537/Eco_RAST_Prokka_BlastKOALA_PTools_DeepEC_DeepGO"
}


{<modelseedpy.core.annotationontology.AnnotationOntologyTerm object at 0x7f8ad03d22b0>: [{'event': 'RAST annotation', 'term': 'SSO:000025637', 'ontology': 'SSO', 'probability': 1.0}]}
{<modelseedpy.core.annotationontology.AnnotationOntologyTerm object at 0x7f8ad03d23a0>: [{'event': 'RAST annotation', 'term': 'SSO:000001104', 'ontology': 'SSO', 'probability': 0.5}], <modelseedpy.core.annotationontology.AnnotationOntologyTerm object at 0x7f8ad03d2400>: [{'event': 'RAST annotation', 'term': 'SSO:000003485', 'ontology': 'SSO', 'probability': 0.5}]}
{<modelseedpy.core.annotationontology.AnnotationOntologyTerm object at 0x7f8ad03d2610>: [{'event': 'RAST annotation', 'term': 'SSO:000003486', 'ontology': 'SSO', 'probability': 1.0}]}
{<modelseedpy.core.annotationontology.AnnotationOntologyTerm object at 0x7f8ad03d2460>: [{'event': 'RAST annotation', 'term': 'SSO:000008061', 'ontology': 'SSO', 'probability': 1.0}]}
{<modelseedpy.core.annotationontology.AnnotationOntologyTerm object at 0x7f8ad03d

4140

In [3]:
#Pulling super annotated ecoli genome
genome_ref = "77537/Eco_RAST_Prokka_BlastKOALA_PTools_DeepEC_DeepGO"
#Link to AnnotationOntology code in github:
annoont = AnnotationOntology.from_kbase_data(annoapi.get_annotation_ontology_events({
    "input_ref" : genome_ref
}),genome_ref)
#You will call the get_reaction_gene_hash function of annoont to get the reaction gene hash
#Pulling ecoli model
model = msrecon.get_model("151253/GCF_000005845.2.RAST.NewGMM.mdl")
tests = model.get_atp_tests()
#Loading transcriptome
expression = pd.read_csv("ExpressionData.tsv",sep="\t")
#Getting template
template = msrecon.get_template(model.model.template_ref)
#Stripping out all reactions from template that are not in model except the reactions we want to keep to test exclusion
remove_list = []
#Keeping reactions we know are rejected because they break this model
keep_list = [
"rxn10157_c",
"rxn10155_c",
"rxn12575_c",
"rxn00704_c",
"rxn00778_c",
"rxn05206_c",
"rxn09193_c",
"rxn08234_c",
"rxn20583_c",
"rxn00154_c",
"rxn00145_c"
]
#rxn05289_c0
#rxn05469_c0
#rxn05654_c0
for rxn in template.reactions:
    if rxn.id+"0" not in model.model.reactions and rxn.id not in keep_list:
        remove_list.append(rxn)
template.remove_reactions(remove_list)
#Intentionally removing reactions related to our test media
#Serine,Malate,Adenosine,Fructose,Pyruvate
remove_ids = ["rxn05307_c0","rxn05605_c0","rxn05318_c0","rxn05560_c0","rxn05469_c0"]
remove_list = []
for item in remove_ids:
    rxn = model.model.reactions.get_by_id(item)
    remove_list.append(rxn)
model.model.remove_reactions(remove_list)
#Making reaction scores
scores = annoont.get_reaction_gene_hash()
kbdevutil.save("reaction_scores",scores)
#Getting MSGapfill object
msgapfill = MSGapfill(
    model,
    [template],
    [],
    tests,
    blacklist=[],
    default_target="bio1",
    minimum_obj=0.01
)
serine_media = msrecon.get_media("KBaseMedia/Carbon-L-Serine")
malate_media = msrecon.get_media("KBaseMedia/Carbon-L-Malic-Acid")
adenosine_media = msrecon.get_media("KBaseMedia/Carbon-Adenosine")
fuctose_media = msrecon.get_media("KBaseMedia/Carbon-D-Fructose")
pyruvate_media = msrecon.get_media("KBaseMedia/Carbon-Pyruvic-Acid")
combined_media = msrecon.get_media("151253/CombinedMedia")
msgapfill.run_multi_gapfill([serine_media,malate_media,adenosine_media,fuctose_media,pyruvate_media,combined_media],target="bio1")


1707167034.834733 INFO: metabolites 1517
1707167035.1896698 INFO: reactions 1650
1707167036.034456 INFO: Default biomass: [bio1]
1707167036.090732 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.
1707167036.2885098 WARNING: Ignoring reaction 'rxn00062_c0' since it already exists.


Getting tests from attributes
PREFILTERING WITH 5 GROWTH CONDITIONS
Failed:<rxn10157_c0
Failed:<rxn10155_c0
Failed:>rxn12575_c0
Failed:<rxn00145_c0
Failed:>rxn00704_c0
Failed:>rxn00778_c0
Failed:>rxn05206_c0
Failed:>rxn09193_c0
Failed:<rxn08234_c0
Failed:>rxn20583_c0


1707167048.868472 INFO: Expansion time:Ac.NO2:1.4932716519997484
1707167048.869034 INFO: Filtered count:10 out of 1088
1707167049.21513 INFO: Expansion time:Ac.NO3:0.3456262149993563
1707167049.215808 INFO: Filtered count:10 out of 1088
1707167049.535228 INFO: Expansion time:Ac.O2:0.3190064139998867
1707167049.535767 INFO: Filtered count:10 out of 1088
1707167049.8277912 INFO: Expansion time:Akg.O2:0.29161893099990266
1707167049.828335 INFO: Filtered count:10 out of 1088


Failed:<rxn00154_c0


1707167051.157321 INFO: Expansion time:Dlac:1.3285555479997129
1707167051.1578822 INFO: Filtered count:11 out of 1088
1707167051.447741 INFO: Expansion time:Dlac.O2:0.28940889200021047
1707167051.448312 INFO: Filtered count:11 out of 1088
1707167051.7200232 INFO: Expansion time:Etho.O2:0.2712331530001393
1707167051.720565 INFO: Filtered count:11 out of 1088
1707167052.009865 INFO: Expansion time:For.NO2:0.28888810299940815
1707167052.010468 INFO: Filtered count:11 out of 1088
1707167052.277259 INFO: Expansion time:For.NO3:0.26638346199979424
1707167052.277909 INFO: Filtered count:11 out of 1088
1707167052.619224 INFO: Expansion time:For.O2:0.340833281000414
1707167052.619905 INFO: Filtered count:11 out of 1088
1707167052.918405 INFO: Expansion time:Fum:0.29804582500037213
1707167052.919083 INFO: Filtered count:11 out of 1088
1707167053.2522979 INFO: Expansion time:Fum.O2:0.3325661160006348
1707167053.253116 INFO: Filtered count:11 out of 1088
1707167053.561124 INFO: Expansion time:Glc:

...

1707167095.5266829 INFO: Expansion time:Ac.NO2:0.2543594539993137
1707167095.527215 INFO: Filtered count:0 out of 1
1707167095.715167 INFO: Expansion time:Ac.NO3:0.1876125700000557
1707167095.7160552 INFO: Filtered count:0 out of 1
1707167095.914546 INFO: Expansion time:Ac.O2:0.19798914900002273
1707167095.915166 INFO: Filtered count:0 out of 1
1707167096.1311848 INFO: Expansion time:Akg.O2:0.21560950799994316
1707167096.1317492 INFO: Filtered count:0 out of 1
1707167096.954977 INFO: Expansion time:Dlac:0.8227594549998685
1707167096.955692 INFO: Filtered count:0 out of 1
1707167097.1728919 INFO: Expansion time:Dlac.O2:0.21667151100064075
1707167097.173478 INFO: Filtered count:0 out of 1
1707167097.348875 INFO: Expansion time:Etho.O2:0.17497587100024248
1707167097.349473 INFO: Filtered count:0 out of 1
1707167097.547677 INFO: Expansion time:For.NO2:0.19776678400012315
1707167097.5482082 INFO: Filtered count:0 out of 1
1707167097.745546 INFO: Expansion time:For.NO3:0.19682815699979983
17

Initial solution: {'reversed': {}, 'new': {'rxn05560_c0': '>'}, 'media': <cobrakbase.core.kbasebiochem.media.Media object at 0x7fe7cc4e2f10>, 'target': 'bio1', 'minobjective': 0.01, 'binary_check': False}
Unneeded: [['rxn05307_c0', '>', 'new', 100], ['rxn05605_c0', '>', 'new', 100], ['rxn05318_c0', '>', 'new', 100]]
Growth: 0.5530460984995947 Carbon-D-Fructose
Adding gapfilling {'growth': 0.5530460984995947, 'media': <cobrakbase.core.kbasebiochem.media.Media object at 0x7fe7cc4e2f10>, 'target': 'bio1', 'minobjective': 0.01, 'binary_check': False, 'new': {'rxn05560_c0': '>'}, 'reversed': {}}


1707167105.449898 INFO: Expansion time:Ac.NO2:0.24827695600015431
1707167105.450594 INFO: Filtered count:0 out of 1
1707167105.624607 INFO: Expansion time:Ac.NO3:0.17355705800036958
1707167105.6252391 INFO: Filtered count:0 out of 1
1707167106.4363542 INFO: Expansion time:Ac.O2:0.8106820650000373
1707167106.437032 INFO: Filtered count:0 out of 1
1707167106.60933 INFO: Expansion time:Akg.O2:0.17182934599986766
1707167106.609942 INFO: Filtered count:0 out of 1
1707167106.825929 INFO: Expansion time:Dlac:0.21558326499962277
1707167106.826454 INFO: Filtered count:0 out of 1
1707167107.029821 INFO: Expansion time:Dlac.O2:0.20295897600044555
1707167107.0305028 INFO: Filtered count:0 out of 1
1707167107.223352 INFO: Expansion time:Etho.O2:0.19239431299956777
1707167107.223951 INFO: Filtered count:0 out of 1
1707167107.425837 INFO: Expansion time:For.NO2:0.20138674900044862
1707167107.4265091 INFO: Filtered count:0 out of 1
1707167107.605071 INFO: Expansion time:For.NO3:0.17806782800016663
170

Initial solution: {'reversed': {}, 'new': {'rxn05469_c0': '>'}, 'media': <cobrakbase.core.kbasebiochem.media.Media object at 0x7fe7cc4e2a60>, 'target': 'bio1', 'minobjective': 0.01, 'binary_check': False}
Unneeded: [['rxn05307_c0', '>', 'new', 100], ['rxn05605_c0', '>', 'new', 100], ['rxn05318_c0', '>', 'new', 100], ['rxn05560_c0', '>', 'new', 100]]
Growth: 0.21652276497861256 Carbon-Pyruvic-Acid
Adding gapfilling {'growth': 0.21652276497861256, 'media': <cobrakbase.core.kbasebiochem.media.Media object at 0x7fe7cc4e2a60>, 'target': 'bio1', 'minobjective': 0.01, 'binary_check': False, 'new': {'rxn05469_c0': '>'}, 'reversed': {}}


{<cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2d90>: {'growth': 0.21792055797325602,
  'media': <cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2d90>,
  'target': 'bio1',
  'minobjective': 0.01,
  'binary_check': False,
  'new': {'rxn05307_c0': '>'},
  'reversed': {}},
 <cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2b80>: {'growth': 0.28174046527337904,
  'media': <cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2b80>,
  'target': 'bio1',
  'minobjective': 0.01,
  'binary_check': False,
  'new': {'rxn05605_c0': '>'},
  'reversed': {}},
 <cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2a00>: {'growth': 0.5547647802983801,
  'media': <cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2a00>,
  'target': 'bio1',
  'minobjective': 0.01,
  'binary_check': False,
  'new': {'rxn05318_c0': '>'},
  'reversed': {}},
 <cobrakbase.core.kbasebiochem.media.Media at 0x7fe7cc4e2f10>: {'growth': 0.5530460984995947,
  'media': <cobrakbase.core.kbasebiochem.media

# Code cell to use to run pipeline on all of Aimee's genomes

In [2]:
#Run full pipeline on https://narrative.kbase.us/narrative/167956
model = msrecon.get_model("151253/GCF_000005845.2.RAST.NewGMM.mdl")

1706332274.054497 INFO: metabolites 1517
1706332274.545055 INFO: reactions 1650
1706332275.494679 INFO: Default biomass: [bio1]


In [11]:
remove_ids = ["rxn05307_c0","rxn05605_c0","rxn05318_c0","rxn05560_c0","rxn05469_c0"]
remove_list = []
for item in remove_ids:
    rxn = model.model.reactions.get_by_id(item)
    remove_list.append(rxn)
model.model.remove_reactions(remove_list)
#Serine,Malate,Adenosine,Fructose,Pyruvate
media = msrecon.get_media("KBaseMedia/Carbon-L-Serine")
model.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print("Serine:"+str(model.model.slim_optimize()))
media = msrecon.get_media("KBaseMedia/Carbon-L-Malic-Acid")
model.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print("Malate:"+str(model.model.slim_optimize()))
media = msrecon.get_media("KBaseMedia/Carbon-Adenosine")
model.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print("Adenosine:"+str(model.model.slim_optimize()))
media = msrecon.get_media("KBaseMedia/Carbon-D-Fructose")
model.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print("Fructose:"+str(model.model.slim_optimize()))
media = msrecon.get_media("KBaseMedia/Carbon-Pyruvic-Acid")
model.pkgmgr.getpkg("KBaseMediaPkg").build_package(media)
print("Pyruvate:"+str(model.model.slim_optimize()))


Serine:0.0
Malate:0.0
Adenosine:0.0
Fructose:0.0
Pyruvate:0.0


# Code for creating ATP tests and checking changes in ATP test outcome

In [ ]:
#Pulling ecoli model
model = msrecon.get_model("151253/GCF_000005845.2.RAST.NewGMM.mdl")
existing_tests = model.get_atp_tests()
recompute_tests = model.get_atp_tests(core_template=msrecon.core_template,recompute=True)
existing_hash = {}
for test in existing_tests:
    existing_hash[test["media"].id] = test
recompute_hash = {}
for test in recompute_tests:
    recompute_hash[test["media"].id] = test
    if test["media"].id in existing_hash:
        if test["threshold"] != existing_hash[test["media"].id]["threshold"]:
            print("Threshold mismatch:"+str(test["threshold"])+";"+str(existing_hash[test["media"].id]["threshold"]))
    else:
        print("Extra recompute test:"+test["media"].id)
for test in existing_tests:
    if test["media"].id not in recompute_hash:
        print("Extra existing test:"+test["media"].id)


# Code for checking if a model and a gapfilling model pass tests

In [ ]:
#Pulling super annotated ecoli genome
genome_ref = "77537/Eco_RAST_Prokka_BlastKOALA_PTools_DeepEC_DeepGO"
#Link to AnnotationOntology code in github:
annoont = AnnotationOntology(annoapi.get_annotation_ontology_events({
    "input_ref" : genome_ref
}),genome_ref)
#You will call the get_reaction_gene_hash function of annoont to get the reaction gene hash
#Pulling ecoli model
model = msrecon.get_model("151253/GCF_000005845.2.RAST.NewGMM.mdl")
tests = model.get_atp_tests()
#Loading transcriptome
expression = pd.read_csv("ExpressionData.tsv",sep="\t")
#Getting template
template = msrecon.get_template(model.model.template_ref)
#Stripping out all reactions from template that are not in model except the reactions we want to keep to test exclusion
remove_list = []
#Keeping reactions we know are rejected because they break this model
keep_list = [
"rxn10157_c",
"rxn10155_c",
"rxn12575_c",
"rxn00704_c",
"rxn00778_c",
"rxn05206_c",
"rxn09193_c",
"rxn08234_c",
"rxn20583_c",
"rxn00154_c",
"rxn00145_c"
]
#rxn05289_c0
#rxn05469_c0
#rxn05654_c0
for rxn in template.reactions:
    if rxn.id+"0" not in model.model.reactions and rxn.id not in keep_list:
        remove_list.append(rxn)
template.remove_reactions(remove_list)
#Intentionally removing reactions related to our test media
#Serine,Malate,Adenosine,Fructose,Pyruvate
remove_ids = ["rxn05307_c0","rxn05605_c0","rxn05318_c0","rxn05560_c0","rxn05469_c0"]
remove_list = []
for item in remove_ids:
    rxn = model.model.reactions.get_by_id(item)
    remove_list.append(rxn)
#model.model.remove_reactions(remove_list)
#Making reaction scores
scores = annoont.get_reaction_gene_hash()
#Getting MSGapfill object
msgapfill = MSGapfill(
    model,
    [template],
    [],
    tests,
    blacklist=[],
    default_target="bio1",
    minimum_obj=0.01
)
print("Model test!")
model.test_condition_list(tests, model=model.model,positive_growth=[])
print("Gapfill model test!")
msgapfill.gfpkgmgr.getpkg("GapfillingPkg").reset_objective_minimum(0,False)
msgapfill.gfmodelutl.test_condition_list(tests,positive_growth=[])

# Code for comparing reaction content in two models and printing differences

In [ ]:
#oldmodel = msrecon.get_model("151253/GCF_000005845.2.RAST.NewGMM.mdl")
oldmodel = model
for rxn in oldmodel.model.reactions:
    if rxn.id[0:3] != "EX_" and rxn.id in msgapfill.gfmodel.reactions:
        if rxn.upper_bound != msgapfill.gfmodel.reactions.get_by_id(rxn.id).upper_bound:
            print(rxn.id,"Upper",rxn.upper_bound,msgapfill.gfmodel.reactions.get_by_id(rxn.id).upper_bound)
        if rxn.lower_bound != msgapfill.gfmodel.reactions.get_by_id(rxn.id).lower_bound:
            print(rxn.id,"Lower",rxn.lower_bound,msgapfill.gfmodel.reactions.get_by_id(rxn.id).lower_bound)
    elif rxn.id[0:3] != "EX_":
        print(rxn.id,rxn.direction,"missing new model")
for rxn in msgapfill.gfmodel.reactions:
    if rxn.id not in oldmodel.model.reactions:
        print(rxn.id," missing old model")